# Preparing to run biomolecular with AMBERTOOLS and then QM/MM simulations with CP2K 



# Preparing to run biomolecular with AMBERTOOLS 

# Overview of AmberTools

AmberTools suite is a set of several independent packages developed as part of the Amber22 software package and distributed free of charge. Its components are mostly released under the GNU General Public License (GPL). They work well by themselves, and with Amber22 itself. The list of packages is

1.NAB/sff: a program for building molecules, running MD or applying distance geometry restraints using generalized Born, Poisson-Boltzmann or 3D-RISM implicit solvent models

2.antechamber and MCPB: programs to create force fields for general organic molecules and metal centers

3.tleap and parmed: basic preparatory tools for Amber simulations

4.sqm: semiempirical and DFTB quantum chemistry program

5.pbsa: Several linear and non-linear numerical solvers for Poisson-Boltzmann models

6.3D-RISM: solves integral equation models for solvation

7.sander: workhorse program for molecular dynamics simulations

8.gem.pmemd: tools for using advanced force fields

9.mdgx: a program for pushing the boundaries of Amber MD, primarily through parameter fitting. Also includes customizable virtual sites and explicit solvent MD capabilities.

10.cpptraj and pytraj: tools for analyzing structure and dynamics in trajectories

11.MMPBSA.py: energy-based analyses of MD trajectories


AmberTools package has multiple tools to prepare, simulate and analyse biological systems.
You should familiarise yourself with tleap, pamred, nab, cpptraj, antechamber, parmch2 and sander as they will ususally be helpful to set up a biological system.

1.tleap is used to generate topologies and coordinates for biological molecules.

2.cpptraj is used to postprocess and analyse AMBER trajectories.

3.antechamber is used to create parameters for non-standard aminoacids and drug-like molecules.

4.sander runs molecular dynamics simulations

5.parmed allows modifying AMBER topologies

# Preparing your PDB file

We are going to use the X-ray structure of Mpro protein bound to a ligand from the PanDDA library. The PDB id we are going to use is 5R84. If you open this entry from the Protein DataBank you will find a lot of information of the protein structure:

1. Experimental Data Snapshot
2. wwPDB Validation
3. Literature
4. Macromolecules
5. Small Molecules
6. Experimental Data & Validation
7. Entry History

The sections Macromolecules and Small Molecules give us information about the contents of the PDB file. We can see that there is only one macromolecule (only one chain) that corresponds to SARS-CoV-2 main protease. There are two small molecules in the structure: DMS and GWS.
DMS is a crystallisation product and it doesn’t interest us.
GWS is the fragment we want to model.
Once we know which molecules from the PDB file we want to model and which should be deleted from the PDB file, we can continue. You can download the PDB file from the website yourself or use the one we are providing in this repository.

In [ ]:
!grep -v -e "GWS" -e "DMS" -e "CONECT" -e "HOH" 5r84.pdb >  protein.pdb

In [ ]:
!grep "GWS" 5r84.pdb > GWS.pdb

In [ ]:
!grep "HOH" 5r84.pdb > waters.pdb

# Replicating the experimental/target conditions



It is important to take into account experimental conditions of the system we want to simulate such as the pH, salinity… Since Mpro protein from SARS-CoV2 is located in the cytosol of the infected human cell, we are going to assume a pH=7.4. Do not asume that will always be the case in the future, you should always, please check the literature on your target protein before you start running your simulations.



# Assessing the protonation states of the protein and preparing the protein for AmberTools



There are several methods to assess the protonation state of the residues in a protein (for further information, here). For example, the H++ server and PROPKA software can help you in the future. We are only going to use PROPKA command-line in this workshop.

PROPKA returns the calculated pKa values for tritable residues in the protein. These are aminoacids, whose sidechain contains a chemical moiety that can change its protonation state depending on the pH of the environment they are found in. You have acidic residues (typically ASP and GLU, but in extreme environments TYR, SER and CYS can act as acids) and basic residues (LYS, ARG and HIS).

Protein forcefield encode these protonation states as different residue names (i.e. GLU and GLH for the two states of glutamic acid). You can find more information about these residues, and how its pKa depends on the chemical environment here .

When preparing a protein, we have to compare each one of the predicted pKa values with the chosen pH: if the pKa is below the chosen pH the residue should be deprotonated (that is acidic residues will be negatively charged and basic residues will be neutral), if above it will be protonated (acidic residues will be neutral and basic residues will be positively charged).

In [ ]:
! propka3 protein.pdb 

# Histidine
Histidines are tricky residues, they have three possible protonation states: HID, HIE and HIP. HIP is the protonated residue. HID and HIE correspond to the two natural tautomers of the neutral histidine, where the proton can be found in delta or epsilon positions. In solution, the most common conformer is HIE but always visualise your structure before assuming any histidine protonation state.

# Converting PDB to Amber file format

Last but not least, we need to clean the PDB file before passing it to AmberTools (removing residues with double ocupation, checking for disulfide bonds, …). So we are going to use the first AmberTools tool: pdb4amber which prepares your PDB file.

In [ ]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/pdb4amber -i protein.pdb -o protein4amber.pdb

The output above gives you a lot of information. First it tells you that your protein is composed of a single chain. Then it informs you that some residues had alternate conformations in the X-ray structure, but that only the first occurrence for each atom was kept.

pdb4amber informs us about the duplicity of several sidechains, disulphide bonds and missing heavy atoms. However, it does not check protonation states for protein residues and therefore we ALWAYS must run a pKa calculation first and apply the required changes to the PDB.

# Parameterising your ligands

Generate parameters for GWS ligand using antechamber and parmchk2.

In this tutorial, we will make use of the antechamber and parmchk2 tools from AmberTools package to generate AMBER force field parameters. We will use the “general AMBER force field 2 (GAFF2)” parameters. This force field was designed to provide atom types and atom parameters needed to parameterise most pharmaceutical molecules, and is compatible with the traditional AMBER force fields for proteins. To assign partial charges, we are going to use AM1-BCC2 charge method. It is an inexpensive and fast method for calculating partial charges and be we don’t need to worry too much about its limitations as partial charges won’t be used during production runs due to the QM treatment of the ligand in the QM/MM simulation.

antechamber allows the rapid generation of topology files for use with the AMBER simulation programs. It is a higher-level wrapper around other AmberTools programs (sqm, divcon, atomtype, am1bcc, bondtype, espgen, respgen and prepgen) and is able to:

1. Automatically identify bond and atom types
2. Judge atomic equivalence
3. Generate residue topology files
4. Find missing force field parameters and supply reasonable suggestions

We will use antechamber to assign atom types to the GWS ligand and calculate a set of point charges. We will have to specify GAFF2 force field (-at gaff2, more information on the parameters here: $AMBERHOME/dat/leap/parm/gaff2.dat), the AM1-BCC2 charge method (-c bcc), the net charge of the molecule (-nc 0) and the name of the new residue generated (We are going to keep GWS: -rn GWS). We are going to output a mol2-type file (-fo mol2) containing the atomtypes and the point charges.

In [ ]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/antechamber -i GWS.H.pdb -fi pdb -o GWS.mol2 -fo mol2 -c bcc -nc 0 -rn GWS -at gaff2

This step generates a lot of intermediate files (in capital letters). You can safely delete them as these are used in antechamber and we no longer require them. The sqm.xxx files are input and output from the sqm quantum mechanics code used by antechamber to calculate the atomic point charges. We are not interested in the data here except to check that the sqm calculation completed successfully.

In [3]:
! tail sqm.out

  QMMM:    28       28      H        2.4540   -0.7994   -1.9818
  QMMM:    29       29      H        0.1680   -0.0745    0.6032
  QMMM:    30       30      H       -5.0099    2.3010   -0.3234
  QMMM:    31       31      H       -4.1135    1.2813    1.8135
  QMMM:    32       32      H       -1.7988    0.3489    1.8379
  QMMM:    33       33      H       -1.4809    1.5305   -2.3735
  QMMM:    34       34      H        3.1030    1.6086   -1.2938

           --------- Calculation Completed ----------



Once we are sure that the point charges were generated The MOL2 file contains tridimensional information of the molecule as well as atom type and point charges. You can find more information on the MOL2 format here. The resulting MOL2 file is shown below:

In [5]:
! cat GWS.mol2

@<TRIPOS>MOLECULE
GWS
   34    35     1     0     0
SMALL
bcc


@<TRIPOS>ATOM
      1 N           -3.2590     2.8360     0.0000 nb         0 GWS      -0.643000
      2 C           -4.5930     2.0660     0.0000 ca         0 GWS       0.363200
      3 C1          -4.5930     0.5260     0.0000 ca         0 GWS      -0.219300
      4 C2          -3.2590    -0.2440     0.0000 ca         0 GWS      -0.156000
      5 C3          -1.9260     0.5260     0.0000 ca         0 GWS      -0.046700
      6 C4          -1.9260     2.0660     0.0000 ca         0 GWS       0.374200
      7 N1          -0.5920    -0.2440     0.0000 ns         0 GWS      -0.467100
      8 C5           0.7420     0.5260     0.0000 c          0 GWS       0.667100
      9 C6           2.0750    -0.2440     0.0000 c3         0 GWS      -0.150400
     10 C7           3.4090     0.5260     0.0000 c3         0 GWS      -0.062700
     11 C8           4.7430    -0.2440     0.0000 c3         0 GWS      -0.075900
     12 C9          

The GAFF and GAFF2 atom types are in lower case. This is the mechanism by which the GAFF force field is kept independent from the macromolecular AMBER force fields. All of the traditional AMBER force fields use uppercase atom types. In this way the GAFF and traditional force fields can be mixed in the same calculation.


# Looking for missing force field parameters with parmchk2

While the most likely combinations of bond, angle and dihedral parameters are defined in the parameter file it is possible that our molecule might contain combinations of atom types for bonds, angles or dihedrals that have not been parameterised. If this is the case, we will have to specify any missing parameters before we can create our prmtop and inpcrd files in LEap.

We will use parmchk2 to test if all the parameters we require are available.

In [1]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/parmchk2 -i GWS.mol2 -f mol2 -o GWS.frcmod -s gaff2

parmchk2 generates a parameter file that can be loaded into Leap in order to add missing parameters. It contains all of the missing parameters. If possible, it will fill in these missing parameters by analogy to a similar parameter. Let’s look at the generated GWS.frcmod file:

In [2]:
! cat GWS.frcmod

Remark line goes here
MASS

BOND
ca-ns  315.20   1.412       same as ca- n, penalty score=  0.0
c -ns  356.20   1.379       same as  c- n, penalty score=  0.0
hn-ns  527.30   1.013       same as hn- n, penalty score=  0.0

ANGLE
ca-ca-ns   85.600     120.190   same as ca-ca-n , penalty score=  0.0
c -ns-ca   65.700     123.710   same as c -n -ca, penalty score=  0.0
ca-ns-hn   48.000     116.000   same as ca-n -hn, penalty score=  0.0
c3-c -ns   84.300     115.180   same as c3-c -n , penalty score=  0.0
ns-c -o   113.800     123.050   same as n -c -o , penalty score=  0.0
c -ns-hn   48.700     117.550   same as c -n -hn, penalty score=  0.0

DIHE
ca-ca-ns-c    1    0.950       180.000           2.000      same as c -n -ca-ca, penalty score=  0.0
ca-ca-ns-hn   4    1.800       180.000           2.000      same as X -ca-n -X , penalty score=  0.0
c3-c -ns-ca   1    0.750       180.000          -2.000      same as c3-c -n -ca
c3-c -ns-ca   1    0.500         0.000           3.000      sam

# TIP
You should check these parameters carefully before running a simulation. If antechamber can’t empirically calculate a value or has no analogy it will either add a default value that it thinks is reasonable or alternatively insert a place holder (with zeros everywhere) and the comment “ATTN: needs revision”. In this case you will have to manually parameterise this yourself. See the links at the beginning of the section.

#Key points

1. Generating parameters for non-conventinal residues is not easy and there are different ways to parameterise them.
2. The quality of the parameters is important to obtain valid results.
3. antechamber creates parameters for a non-conventional residues.
4. ALWAYS check the parameters for your system and test that they behave as expected

# System set up

Generate topology and coordinate files with tleap

Once we have parameters for our ligand and have prepared a protein PDB file, we can start building our system and creating topology and coordinate files for AMBER. To create the mentioned files we will use LEaP. there are two versions of leap: the GUI version xLEaP and the terminal version tLEaP. xLEaP is better to learn Leap because you can visualise the result of each command you try. However, xLEaP is quite buggy and it can be frustrating before you get used to all its quirks.

In this session, we are going to use tleap and I am going to walk you through every command we use. If you type tleap in your terminal you should enter in the tLEaP command-line. If you want to check what you did in previous session of LEaP, LEaP is quite verbose and informs you of everything it does as well as writing it into the leap.log file.

We have prepared a LEaP script to set up our system, which we are going to explain briefly before executing it.

We are going to use default force fields (amberff14SB, tip3p and GAFF2) in this tutorial, so we can use the leaprc scripts already included in AMBER to load them into tleap:

leaprc.protein.ff14SB for the protein
leaprc.gaff2 for the ligand molecule
leaprc.water.tip3p for the water molecules.

# Load force field parameters
source leaprc.protein.ff14SB
source leaprc.gaff2
source leaprc.water.tip3p

Now we have to load our three units (protein, ligand and water molecules) into tLeap. We are going to start with the ligand by lading the two files we have created before: GWS.frcmod and GWS.mol2 (In this case the mol2 file already includes the correct coordinates of the ligand). After, we load the coordinates for the protein and the water molecules with the loadPB command.

# Load extraparameters for the ligand
loadamberparams GWS.frcmod

# Load protein, ligand and water molecules
protein = loadPDB protein4amber.pdb
GWS = loadmol2 GWS.mol2
waters = loadPDB waters.pdb

From the very beginning of this series, we have split the system into these three pieces, now we join them again with the combine command.

# Build system
system = combine {GWS protein waters}
savepdb system system.dry.pdb

check system

It is good practice to check the resulting structure of this combination step (savePDB command to save a PDB file) and to perform a sanity check on the unit we have created by using the check command: this determines the charge of the system, flags up close contacts (it is normal to have them after adding hydrogen atoms), and checks if there are any missing parameters.


If the unit is OK, we can continue! Now we need to solvate it and add counterions to neutralise the system

To solvate the system, we can use 2 different commands: solvateOct and solvateBox. These commands differ in the shape of the resulting simulation box: the first command creates a truncated octahedron solvation box and the second a cubic solvation box. Despite truncated octahedric boxes being smaller and compatible with globular proteins, we are going to use solvateBox because CP2K only allows orthorombic boxes.

Last but not least, we need to neutralise the system by adding counterions. To this end, we will add Na+ and Cl- ions until our box reaches the correct charge. This is done with the addions2 command.

# Solvate
solvateBox system TIP3PBOX 10 iso

# Neutralise
addions2 system Cl- 0
addions2 system Na+ 0

Now we can proceed to save our topology and coordinate files with saveamberparm system system.parm7 system.rst7. Before doing anything else, LEaP checks that the unit it is OK and then proceeds to build the topology and write the coordinates. Finally, it prints a summary of the system molecules. We have a ligand named GWS and 13121 water molecules. It is important that you include the saveAmberParm command line, otherwise the information we have loaded will be lost once we exit LEap.

# Save AMBER input files
savePDB system system.pdb
saveAmberParm system system.parm7 system.rst7

quit
We have provided all these commands in a leapin file. To execute it you should type the following:

tleap -f leapin

#key Points

LEap creates topologies and initial coordinates in AMBER format.
LEap can generate initial coordinates from scratch if needed.
xLEap (with GUI) and tLEap (through the terminal) are the two flavours of LEap.
Your MM system should always be neutral, add counterions to neutralise your system.
ALWAYS visualise and check the structures created by LEap.

# Energy minimisation

We are going to minimise and equilibrate the system using the sander tool from AmberTools suite. We have provided commented input files and ARCHER submission scripts to make these steps easier to follow. You can find more information on the sander input options in Chapter 19 of the Amber20 manual.

To run sander, one needs to specify at least the initial coordinates (-c system.rst7) and the topology files of your system (-p system.parm7) and an input file (-i sander_min.in) providing the details of the simuilation to perform. Also, we can specify the name of output files such the log file (-o) and final coordinates (-r).

# CODE

Minimisation of system 
 &cntrl
  imin=1,        ! Perform an energy minimization.
  maxcyc=4000,   ! The maximum number of cycles of minimization. 
  ncyc=2000,     ! The method will be switched from steepest descent to conjugate gradient after NCYC cycles.
 /


# Energy minimisation with sander

We have devised a minimisation protocol that will perform 4000 steps using two different methods: 2000 of gradient descent and 2000 of conjugate gradient. The sander input file looks like this:

Minimisation of system 
 &cntrl
  imin=1,        ! Perform an energy minimization.
  maxcyc=4000,   ! The maximum number of cycles of minimization. 
  ncyc=2000,     ! The method will be switched from steepest descent to conjugate gradient after NCYC cycles.
 /
sander should have produced the following files:

min_classical.out where it has written down the energy of the system every 50 steps.
system.min.rst7 with the minimised coordinates.

# Key Points

After adding water molecules and combining coordinates of different system elements we have to minimise the system to fix any bad contacts (LEap had warned us already of some bad contacts in the structure).
Removing bad contacts at this point will prevent our system from failing catastrophically later down the line.
Combining different minimisation methods is a good practice and can help to avoid getting stuck into a local minima.

# Thermalisation and Density equilibration

After the minimisation step, we will heat the system up to the target temperature 298K (25°C, standard value for room temperature experiments) while keeping the volume constant. Afterwards, we will allow the volume to fluctuate and equilibrate the density of the system at a contant pressure of 1 atm (standard value) while still keeping the temperature at 298K.

# Thermalisation

We will use the input file sander_heat.in to gradually increase the temperature of the system up to our target value over the first 30 ps. To help our system accomodate to this change, we will use a temperature ramp in which we control the temperaure increase per timestep.

# CODE
Heating ramp from 0K to 300K 
 &cntrl
  imin=0,                   ! Run molecular dynamics.
  ntx=1,                    ! Initial file contains coordinates, but no velocities.
  irest=0,                  ! Do not restart the simulation, (only read coordinates from the coordinates file)
  nstlim=15000,             ! Number of MD-steps to be performed.
  dt=0.002,                 ! Time step (ps)
  ntf=2, ntc=2,             ! Constrain lengths of bonds having hydrogen atoms (SHAKE)
  tempi=0.0, temp0=298.0,   ! Initial and final temperature
  ntpr=500, ntwx=500,       ! Output options
  cut=8.0,                  ! non-bond cut off
  ntb=1,                    ! Periodic conditiond at constant volume
  ntp=0,                    ! No pressure scaling
  ntt=3, gamma_ln=2.0,      ! Temperature scaling using Langevin dynamics with the collision frequency in gamma_ln (ps−1)
  ig=-1,                    ! seed for the pseudo-random number generator will be based on the current date and time.
  nmropt=1,                 ! NMR options to give the temperature ramp.
 /
&wt type='TEMP0', istep1=0, istep2=12000, value1=0.0, value2=298.0 /
&wt type='TEMP0', istep1=12001, istep2=15000, value1=298.0, value2=298.0 /
&wt type='END' /

# sander produces several output files:

heat_classical.out: Log file with system values stored during the run.
system.heat.rst7: coordinates and velocities to restart the simulation.
system.heat.nc: trajectory in netcdf format of the simulation.

# Pressure equilibration

Once the thermalisation step has finished, sander will use the input file sander_equil.in to equilibrate the density of the system. It is advised to heat and equilibrate over longer periods of simulations than the ones used in this workshop. However for the sake of time, we will use the same simulation length (30ps):

# CODE

Density equilibration
&cntrl
  imin= 0,                       ! Run molecular dynamics.
  nstlim=15000,                  ! Number of MD-steps to be performed.
  dt=0.002,                      ! Time step (ps)
  irest=1,                       ! Restart the simulation and read coordinates and velocities from the restart file provided in -c
  ntx=5,                         ! Initial file contains coordinates and velocities.
  ntpr=500, ntwx=500, ntwr=500,  ! Output options
  cut=8.0,                       ! non-bond cut off
  temp0=298,                     ! Temperature
  ntt=3, gamma_ln=3.0,           ! Temperature scaling using Langevin dynamics with the collision frequency in gamma_ln (ps−1)
  ntb=2,                         ! Periodic conditiond at constant pressure
  ntc=2, ntf=2,                  ! Constrain lengths of bonds having hydrogen atoms (SHAKE)
  ntp=1, taup=2.0,               ! Pressure scaling
  iwrap=1, ioutfm=1,             ! Output trajectory options
/

# Again, sander produces several output files:

equil_classical.out: Log file with thermodynamic data of the system stored during the run.
system.equil.rst7: coordinates and velocities to restart the simulation.
system.equil.nc: trajectory in netcdf format of the simulation.

# Analysis of the MM equilibration simulation

We will now have a closer look at the final steps highlighted before. In the log files, we have monitored the values of several quantities. Especially relevant are:


1. NSTEP: The time step that the MD simulation is at
2. TIME: The total time of the simulation (including restarts)
3. TEMP: System temperature
4. PRESS: System pressure
5. Etot: Total energy of the system
6. EKtot: Total kinetic energy of the system
7. EPtot: Total potential energy of the system


AmberTools provides process_mdout.perl, a perl script to gather and print this information in a easy-to-use format. You can easily call it from the command-line like this:

In [ ]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/process_mdout.perl heat_classical.out equili_classical.out

# Key Points

1. Thermalisation takes our system up to the target temperature.
2. Using a temperature ramp is a good way to slowly increase the system temperature and avoid your system from blowing up due to some bad contacts in your coordinates. You can set up a temperature ramp using the NMR restraint options of sander.
3. Density equilibration fixes the water density and corrects the size simulation box. It prevents air bubbles in our system that might cause simulation artefacts.

# Adding missing LJ parameters and recentering coordinates

1. Find missing parameters in the AMBER forcefields using parmed.
2. Use parmed to change parameters in AMBER7 topologies.
3. Use cpptraj to recenter the system in the simulation box.

# Adding Lennard-Jones parameters using parmed

By construction, some hydrogen atoms do not have Lennard-Jones parameters in the AMBER forcefields. However, before running any QM/MM simulation, we need to provide those parameters. In particular, we are going to add the values of the hydrogen atom that correspond to the hydroxyl in the GAFF forcefield to TIP3P water model and to the hydroxyl groups from serine and tyrosine residues.

To do that, we will use the parmed tool of the AmberTools package. The usage of this program is similar to LEap. First, we load the topology file in parmed and then we do have a lot of options to modify it.

In [ ]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/parmed system.parm7 

To modify them we need to use the changeLJSingleType command on different atom types (:WAT@H1, :SER@HG and :TYR@HH).

changeLJSingleType :WAT@H1 0.3019 0.047

changeLJSingleType :WAT@H2 0.3019 0.047

changeLJSingleType :SER@HG 0.3019 0.047

changeLJSingleType :TYR@HH 0.3019 0.047

It is very important to do this before running your QM/MM simulations. This step prevents creating unphysical interactions between the hydrogen atoms without LJ parameters and the QM subsystem. After this topology correction, the thermostat should stay stable during the QM/MM calculation.

Changing :WAT@H1 Lennard-Jones well depth from 0.0000 to 0.0470 (kal/mol) and radius from 0.0000 to 0.3019 (Angstroms)
Finally, we save a new topology using the command outparm and exit.

outparm system_LJ_mod.parm7

quit

You can also run parmed with an parmed.in input file similarly as we did with tleap.

parmed system.parm7 -i parmed.in

# Centering the solutes in the simulation box
Last but not least, we need to recenter the protein to the center of the simulation box using cpptraj. cpptraj is the trajectory analysis tool of the AmberTools package and it can do a lot of different analysis (you can find more information here). In this tutorial, we are only going to use it to recenter the system and make sure the coordinate file is in the correct format.

First, we need to load the topology file:

In [ ]:
! /Users/nj465/opt/anaconda3/envs/AmberTools22/bin/cpptraj -p system_LJ_mod.parm7 

Then, we provide cpptraj three different commands:

trajin: input trajectories. It can be repeated as many times as trajectory file you want to analyse.
trajout: output trajectories specifying the desired format.
autoimage: recenters protein coordinates and fixes the simulation box around it.
After you have typed all your analysis commands you have to type go or run to actuallt make cpptraj run the analysis.

# Code
trajin system.equil.rst7

autoimage

trajout system.equil0.rst7 inpcrd

go 

quit

Once we have the corrected topology and coordinates, we can start equilibration our QM/MM system. Before you go to the next step, make sure you have the following files:

system_LJ_mod.parm7

system.equil0.rst7

# Key Points

1. In the MM forcefields, hydrogen atoms do not have Lennard-Jones parameters.

2. Adding missing Lennard-Jones parameters to the topology will prevent having unphysical interactions int the QM/MM boundary.

3. You can modify amber topologies using parmed.

4. You should recenter the system to avoid having a split QM region in the QM/MM.

5. cpptraj is the main analysis and postprocess trajectory tool for Amber.

6. Using autoimage in cpptraj recenters the solute and fixes most of the PBC problems of the simulation.